In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import sys
import re
from pyprojroot import here


# Load data

**Markers**

In [2]:
markers = pd.read_csv(here('03_downstream_analysis/05_SPECTRA/results/SPECTRAFactor_Processed_LAM0.001_S0.6_markersDF.csv'), index_col=0)
markers

,0-X-global-X-global_all_IL6-JAK-STAT3_signaling,1-X-global-X-B_IFN_Type_1_2_Lambda,2-X-global-X-global_all_type-I and II-ifn-response,3-X-global-X-global_all_TNF-via-NFkB_signaling,4-X-global-X-global_all_MHC-II-I presentation,5-X-global-X-global_all_complement_production,7-X-B-X-effector,8-X-B-X-B_chemokines,9-X-B-X-Plasma_chemokine_receptors,10-X-B-X-UTC_cytokine_and_receptors_proinflammatory,...,123-X-UTC-X-cDC_antigen_presentation_molecules,125-X-pDC-X-T_CD4_NonNaive_chemokines,126-X-pDC-X-T_CD8_Naive_chemokine_receptors,127-X-pDC-X-T_CD4_Naive_cytokine_and_receptors_proinflammatory,128-X-pDC-X-T_CD8_Naive_cytokine_and receptors_ antiinflammatory,129-X-pDC-X-pDC_IFN_Type_1_2_Lambda,130-X-pDC-X-T_CD8_Naive_IFN_response,131-X-pDC-X-T_CD8_NonNaive_TNF_receptors_ligands,132-X-pDC-X-Plasma_adhesion_molecules,133-X-pDC-X-pDC_antigen_presentation_molecules
ENSG00000004468,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000006327,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
ENSG00000010278,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000026103,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000026508,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235162,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000239713,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000162692,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000095970,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


**varDF**

In [3]:
varDF = pd.read_csv(here("03_downstream_analysis/05_SPECTRA/results/varDF.csv"), index_col=0)
columns_to_remove = ['hgnc_id', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt']
varDF = varDF.drop(columns=columns_to_remove)
varDF.head()

,symbol,locus_group
ensembl_gene_id,,
ENSG00000000003,TSPAN6,protein_coding
ENSG00000000419,DPM1,protein_coding
ENSG00000000457,SCYL3,protein_coding
ENSG00000000460,FIRRM,protein_coding
ENSG00000000938,FGR,protein_coding


# Remove kappa and lambda variable regions

In [4]:
factor_genes = markers.index
factor_genes = factor_genes.map(varDF['symbol'])
len(factor_genes)

1020

In [5]:
pattern = re.compile(r'^IGH(J|V)|IGK(J|V)|^IGL(J|V)')
factor_genes_filtered = [gene for gene in factor_genes if not pattern.match(gene)]
len(factor_genes_filtered)

989

# Save genes for cell-identity check

In [6]:
df = pd.DataFrame({'FactorGenes': factor_genes_filtered})
df

,FactorGenes
0,CD38
1,TNFRSF12A
2,CD9
3,FAS
4,CD44
...,...
984,C12orf75
985,APOBEC3G
986,VCAM1
987,TREM2


In [18]:
df = pd.DataFrame({'FactorGenes': factor_genes_filtered})
df.to_excel(here('03_downstream_analysis/05_SPECTRA/results/markers/filtered_factor_genes.xlsx'), index=False)

# Remove cell-identity genes

In [7]:
file_path = here('03_downstream_analysis/05_SPECTRA/results/markers/filtered_factor_genes_edited.csv')
df = pd.read_csv(file_path,index_col=0)
df

,FactorGenes,ToRemove
0,A2M,0.0
1,ABCA1,0.0
2,ABLIM1,0.0
3,ACKR2,0.0
4,ACKR3,0.0
...,...,...
984,ZEB2,0.0
985,ZFP36L1,0.0
986,ZFP36L2,0.0
987,ZNF385A,0.0


In [8]:
selected_genes = df.loc[df['ToRemove'] == 0, 'FactorGenes'].tolist()
len(selected_genes)

936

In [11]:
subset_varDF = varDF[varDF['symbol'].isin(selected_genes)]
subset_varDF.locus_group.value_counts()

locus_group
protein_coding    928
other               7
pseudogene          1
Name: count, dtype: int64

In [18]:
subset_varDF.query("locus_group == 'other'")

,symbol,locus_group
ensembl_gene_id,,
ENSG00000211890,IGHA2,other
ENSG00000211892,IGHG4,other
ENSG00000211893,IGHG2,other
ENSG00000211895,IGHA1,other
ENSG00000211896,IGHG1,other
ENSG00000211897,IGHG3,other
ENSG00000277734,TRAC,other


In [19]:
subset_varDF = subset_varDF[subset_varDF["locus_group"] != "pseudogene"]
subset_varDF

,symbol,locus_group
ensembl_gene_id,,
ENSG00000001167,NFYA,protein_coding
ENSG00000002549,LAP3,protein_coding
ENSG00000002586,CD99,protein_coding
ENSG00000004468,CD38,protein_coding
ENSG00000005339,CREBBP,protein_coding
...,...,...
ENSG00000276600,RAB7B,protein_coding
ENSG00000277443,MARCKS,protein_coding
ENSG00000277632,CCL3,protein_coding


# Check Factor size

In [20]:
markers_filtered = markers.loc[subset_varDF.index]
markers_filtered

,0-X-global-X-global_all_IL6-JAK-STAT3_signaling,1-X-global-X-B_IFN_Type_1_2_Lambda,2-X-global-X-global_all_type-I and II-ifn-response,3-X-global-X-global_all_TNF-via-NFkB_signaling,4-X-global-X-global_all_MHC-II-I presentation,5-X-global-X-global_all_complement_production,7-X-B-X-effector,8-X-B-X-B_chemokines,9-X-B-X-Plasma_chemokine_receptors,10-X-B-X-UTC_cytokine_and_receptors_proinflammatory,...,123-X-UTC-X-cDC_antigen_presentation_molecules,125-X-pDC-X-T_CD4_NonNaive_chemokines,126-X-pDC-X-T_CD8_Naive_chemokine_receptors,127-X-pDC-X-T_CD4_Naive_cytokine_and_receptors_proinflammatory,128-X-pDC-X-T_CD8_Naive_cytokine_and receptors_ antiinflammatory,129-X-pDC-X-pDC_IFN_Type_1_2_Lambda,130-X-pDC-X-T_CD8_Naive_IFN_response,131-X-pDC-X-T_CD8_NonNaive_TNF_receptors_ligands,132-X-pDC-X-Plasma_adhesion_molecules,133-X-pDC-X-pDC_antigen_presentation_molecules
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000001167,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000002549,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000002586,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
ENSG00000004468,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000005339,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000276600,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000277443,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ENSG00000277632,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False


In [24]:
sorted_column_sums = markers_filtered.sum(axis=0).sort_values()
print(sorted_column_sums[1:40])

45-X-Mono-X-UTC_IFN_Type_1_2_Lambda                                     2
120-X-UTC-X-T_CD8_NonNaive_IFN_response                                 2
5-X-global-X-global_all_complement_production                           5
85-X-T_CD4_NonNaive-X-B_IFN_Type_1_2_Lambda                             7
119-X-UTC-X-ILC_IFN_Type_1_2_Lambda                                     8
129-X-pDC-X-pDC_IFN_Type_1_2_Lambda                                     8
70-X-T_CD4_Naive-X-T_CD4_NonNaive_IFN_Type_1_2_Lambda                   8
97-X-T_CD8_Naive-X-T_CD8_Naive_IFN_Type_1_2_Lambda                      8
23-X-DC-X-T_CD8_Naive_IFN_Type_1_2_Lambda                               8
79-X-T_CD4_NonNaive-X-T_CD4_Naive_CD4T_TH1_UP                           8
62-X-T_CD4_Naive-X-T_CD4_NonNaive_CD4T_TH2_UP                           8
55-X-Plasma-X-Plasma_IFN_Type_1_2_Lambda                                9
33-X-ILC-X-T_CD4_Naive_IFN_Type_1_2_Lambda                              9
12-X-B-X-ILC_IFN_Type_1_2_Lambda      

In [25]:
s = markers.sum(axis=0).sort_values()
print(s[1:40])

45-X-Mono-X-UTC_IFN_Type_1_2_Lambda                                     2
120-X-UTC-X-T_CD8_NonNaive_IFN_response                                 2
5-X-global-X-global_all_complement_production                           5
85-X-T_CD4_NonNaive-X-B_IFN_Type_1_2_Lambda                             7
129-X-pDC-X-pDC_IFN_Type_1_2_Lambda                                     8
119-X-UTC-X-ILC_IFN_Type_1_2_Lambda                                     8
97-X-T_CD8_Naive-X-T_CD8_Naive_IFN_Type_1_2_Lambda                      8
70-X-T_CD4_Naive-X-T_CD4_NonNaive_IFN_Type_1_2_Lambda                   8
23-X-DC-X-T_CD8_Naive_IFN_Type_1_2_Lambda                               8
79-X-T_CD4_NonNaive-X-T_CD4_Naive_CD4T_TH1_UP                           8
62-X-T_CD4_Naive-X-T_CD4_NonNaive_CD4T_TH2_UP                           8
55-X-Plasma-X-Plasma_IFN_Type_1_2_Lambda                                9
33-X-ILC-X-T_CD4_Naive_IFN_Type_1_2_Lambda                              9
12-X-B-X-ILC_IFN_Type_1_2_Lambda      

# Save results

In [33]:
import pickle
selected_genes = markers_filtered.index.tolist()
path = here('03_downstream_analysis/05_SPECTRA/results/SPECTRAFactor_selected_genes.pkl')
with open(path, 'wb') as f:
    pickle.dump(selected_genes, f)

In [28]:
markers_filtered.to_csv(here('03_downstream_analysis/05_SPECTRA/results/SPECTRAFactor_Processed_LAM0.001_S0.6_NonCellIdentitymarkersDF.csv'), index=True)